In [12]:
print("hello world")

hello world


In [13]:
%pwd

'd:\\Projects\\AI-Medical-recommendation'

In [14]:
# import os
# os.chdir("../")

In [15]:
pip show langchain

Name: langchain
Version: 0.3.23
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: d:\Projects\AI-Medical-recommendation\venv\Lib\site-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: langchain-community
Note: you may need to restart the kernel to use updated packages.


In [16]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [17]:
#Extarct data from the Pdf File
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob = "*.pdf", loader_cls = PyPDFLoader)
    documents =loader.load()
    return documents

In [18]:
%pwd

'd:\\Projects\\AI-Medical-recommendation'

In [19]:
extracted_data = load_pdf_file(data = 'Data/')

In [20]:
extracted_data

[Document(metadata={'producer': 'ABBYY FineReader 11', 'creator': 'PDFium', 'creationdate': '2018-05-24T12:11:58+00:00', 'moddate': '2018-05-24T12:11:58+00:00', 'source': 'Data\\Ayurveda-Based-Diet-Lifestyle-Guidelines-Skin-Diseas.pdf', 'total_pages': 48, 'page': 0, 'page_label': '1'}, page_content='Ayurveda based diet & life Style Guidelines \nfor Prevention and Management of Skin Diseases\nCENTRAL COUNCIL FOR RESEARCH IN AYURVEDIC SCIENCES\nMinistry of AYUSH, Government of India  \nNew Delhi'),
 Document(metadata={'producer': 'ABBYY FineReader 11', 'creator': 'PDFium', 'creationdate': '2018-05-24T12:11:58+00:00', 'moddate': '2018-05-24T12:11:58+00:00', 'source': 'Data\\Ayurveda-Based-Diet-Lifestyle-Guidelines-Skin-Diseas.pdf', 'total_pages': 48, 'page': 1, 'page_label': '2'}, page_content='© Central Council for Research in Ayurvedic Sciences \nMinistry of AYUSH, Government of India, New Delhi - 110058\nPublisher: Central Council for Research in Ayurvedic Sciences, Ministry of AYUSH, 

In [21]:
def text_split(extracted_data):
    text_splitter =RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap= 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [22]:
text_chunks = text_split(extracted_data)
print("length of Text Chunks", len(text_chunks))

length of Text Chunks 825


In [23]:
from langchain.embeddings import HuggingFaceEmbeddings

In [24]:
# Download the Embeddings from Hugging Face

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [25]:
embeddings = download_hugging_face_embeddings()

C:\Users\nprat\AppData\Local\Temp\ipykernel_11384\3605476162.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [26]:
query_result = embeddings.embed_query("Hello World")
print("Length ", len(query_result))

Length  384


In [27]:
query_result

[-0.03447727486491203,
 0.03102317824959755,
 0.006734970025718212,
 0.026108985766768456,
 -0.03936202451586723,
 -0.16030244529247284,
 0.06692401319742203,
 -0.006441489793360233,
 -0.0474504791200161,
 0.014758856035768986,
 0.07087527960538864,
 0.05552763119339943,
 0.019193334504961967,
 -0.026251312345266342,
 -0.01010954286903143,
 -0.02694045566022396,
 0.022307461127638817,
 -0.022226648405194283,
 -0.14969263970851898,
 -0.017493007704615593,
 0.00767625542357564,
 0.05435224249958992,
 0.0032543970737606287,
 0.031725890934467316,
 -0.0846213847398758,
 -0.02940601296722889,
 0.05159561336040497,
 0.04812406003475189,
 -0.0033148222137242556,
 -0.058279167860746384,
 0.04196927323937416,
 0.022210685536265373,
 0.1281888335943222,
 -0.022338971495628357,
 -0.011656315997242928,
 0.06292839348316193,
 -0.032876335084438324,
 -0.09122604131698608,
 -0.031175347045063972,
 0.0526994913816452,
 0.04703482985496521,
 -0.08420311659574509,
 -0.030056199058890343,
 -0.02074483036

In [28]:
from dotenv import load_dotenv
load_dotenv()

True

In [29]:
import os

load_dotenv(dotenv_path=".env")  # Ensure the correct path
api_key = os.getenv("DEEPSEEK_API_KEY")
print(api_key)

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


In [30]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
DEEPSEEK_API_KEY = os.environ.get('DEEPSEEK_API_KEY')

In [31]:
from pinecone.grpc import PineconeGRPC as pinecone
from pinecone import ServerlessSpec
import os

pc = pinecone(api_key=PINECONE_API_KEY)

index_name = "aimedication"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'x-cloud-trace-context': '5eb78acebb6b362d9f2a8b53079edf6a', 'date': 'Mon, 07 Apr 2025 10:33:43 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [32]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["DEEPSEEK_API_KEY"] = DEEPSEEK_API_KEY

In [33]:
#Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents = text_chunks,
    index_name=index_name,
    embedding=embeddings,
)


In [34]:
#Load the Existing index
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)


In [35]:
docsearch

In [36]:
retriver = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [37]:
retrieved_docs = retriver.invoke("how to cure thyroid ?")

In [38]:
retrieved_docs

[Document(id='42fff609-ed41-4f6f-9d8d-7e1a61542097', metadata={'creationdate': '2018-05-24T09:09:14+00:00', 'creator': 'PyPDF', 'keywords': 'Device=PressSheet, PageSize=PressSheetSize, PrepsScreening=value', 'moddate': '2018-05-24T09:09:14+00:00', 'page': 23.0, 'page_label': '24', 'producer': 'ABBYY FineReader 11', 'source': 'Data\\Cardiac_disorders.pdf', 'title': 'CCRS Cardiology_BOOK-cd mail.pdf', 'total_pages': 34.0}, page_content='• Timely intake of balanced diet and increase the quantity of fruit and vegetable intake .\n• Weight reduction\n• Timely sleeping and awakening.\n• Regular blood pressure check-up.\n• Regular physical exercise.\n• Daily brisk walking for half an hour.\n• Regular practice of Yoga, Meditation etc. under the supervision of Yoga expert. \nApathya(Don’t% )\n• Excessive intake of salt (sprinkling over salad, curd etc.)'),
 Document(id='c0bc20c9-ad20-4532-854d-4d866c32fb9f', metadata={'creationdate': '2018-05-24T09:09:14+00:00', 'creator': 'PyPDF', 'keywords': '

In [40]:
from langchain_openai import OpenAI
llm = OpenAI(api_key= os.getenv("DEEPSEEK_API_KEY"),
    base_url="https://openrouter.ai/api/v1",temperature=0.4, max_tokens=500)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an AI health and wellness consultant. Provide medically accurate, natural, and science-based recommendations. Only respond to the following topics:"
    "General Health & Fitness – Suggest dietary habits, exercise routines, and lifestyle improvements."
    "Travel Health Precautions – If allergies, food restrictions, weather, or environmental factors are mentioned, recommend:"
    "Suitable fruits, vegetables, herbs, and medications"
    "Hydration tips, breathing exercises, clothing choices"
    "Natural Remedies & Alternative Medicine – Recommend Ayurvedic, herbal, or holistic treatments."
    "Climate & Environmental Adaptation – Guide on adjusting to hot, cold, humid, or dry conditions."
    "Food & Nutrient Advice – Recommend immune-boosting foods, vitamins, and hydration guidance."
    "If the query is outside these areas, respond with:"
    "Sorry, I can't help you with this :("
    "If the query is unclear, ask a clarifying question. Avoid topics like finance, technology, history, politics, or entertainment."
    "\n\n"
    "{context}" 


)



prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [42]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriver, question_answer_chain)

In [44]:
response = rag_chain.invoke({"input": "What is acne ?"})
print(response["answer"])

BadRequestError: Error code: 400 - {'error': {'message': 'Input required: specify "prompt"', 'code': 400}, 'user_id': 'user_2v7WF7le4d7Ob1DEt9xd4tVnb5k'}